## Dataframe operations
Spark DataFrames allow operations similar to pandas Dataframes. We demonstrate some of those.

For more, see the [official guide](https://spark.apache.org/docs/latest/sql-programming-guide.html) and [this article](https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/)

## But first, some important details
Spark on datahub can run in two modes:

* **local mode** which means that it is run on the same computer as the head node. This is convenient for small jobs and for debugging. Can also be done on your laptop.

* **remote mode** in which the head node and the worker nodes are separate. This mode requires that the spark cluster is up and running. In this case the full resources of the clusters are available. This mode is useful when processing large jobs.

In [ ]:
import os
import sys

import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType,BinaryType
import os

os.environ["PYSPARK_PYTHON"]="python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"

In [ ]:
%%time
sc = SparkContext('local', 'test')
sc

CPU times: user 21.4 ms, sys: 10 ms, total: 31.4 ms
Wall time: 5.08 s


<SparkContext master=local appName=test>

In [3]:
%%time

sqlContext = SQLContext(sc)
sqlContext

CPU times: user 3.67 ms, sys: 2.15 ms, total: 5.81 ms
Wall time: 134 ms


/usr/local/spark/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
from os.path import split,join,exists
from os import mkdir,getcwd,remove
from glob import glob

# create directory if needed
notebook_dir=getcwd()
data_dir=join(split(notebook_dir)[0],'Data')
weather_dir=join(data_dir,'Weather')

file_index='NY'
zip_file='%s.tgz'%(file_index)

In [5]:
weather_parquet = join(weather_dir,zip_file[:-3]+'parquet')
print(weather_parquet)
df = sqlContext.read.load(weather_parquet)
df.show(1)

+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
|    Station|Measurement|Year|              Values|       dist_coast|      latitude|         longitude|        elevation|state|             name|
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
|USW00094704|   PRCP_s20|1945|[00 00 00 00 00 0...|361.8320007324219|42.57080078125|-77.71330261230469|208.8000030517578|   NY|DANSVILLE MUNI AP|
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
only showing top 1 row



In [6]:
df.printSchema()

root
 |-- Station: string (nullable = true)
 |-- Measurement: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- Values: binary (nullable = true)
 |-- dist_coast: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- state: string (nullable = true)
 |-- name: string (nullable = true)



In [7]:
%%time
print(df.count())
df.show(1)

168398
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
|    Station|Measurement|Year|              Values|       dist_coast|      latitude|         longitude|        elevation|state|             name|
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
|USW00094704|   PRCP_s20|1945|[00 00 00 00 00 0...|361.8320007324219|42.57080078125|-77.71330261230469|208.8000030517578|   NY|DANSVILLE MUNI AP|
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
only showing top 1 row

CPU times: user 5.49 ms, sys: 858 µs, total: 6.34 ms
Wall time: 1.6 s


### .describe()
The method `df.describe()` computes five statistics for each column of the dataframe `df`.

The statistics are: **count, mean, std, min,max**

You get the following man page using the command `df.describe?`

```
Signature: df.describe(*cols)
Docstring:
Computes statistics for numeric and string columns.

This include count, mean, stddev, min, and max. If no columns are
given, this function computes statistics for all numerical or string columns.

.. note:: This function is meant for exploratory data analysis, as we make no
    guarantee about the backward compatibility of the schema of the resulting DataFrame.

>>> df.describe(['age']).show()
+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|                 2|
|   mean|               3.5|
| stddev|2.1213203435596424|
|    min|                 2|
|    max|                 5|
+-------+------------------+
>>> df.describe().show()
+-------+------------------+-----+
|summary|               age| name|
+-------+------------------+-----+
|  count|                 2|    2|
|   mean|               3.5| null|
| stddev|2.1213203435596424| null|
|    min|                 2|Alice|
|    max|                 5|  Bob|
+-------+------------------+-----+

.. versionadded:: 1.3.1
File:      ~/spark-2.2.1-bin-hadoop2.7/python/pyspark/sql/dataframe.py
Type:      method
```

In [8]:
df.describe().select('Station','Measurement','Year').show() 

+-----------+-----------+------------------+
|    Station|Measurement|              Year|
+-----------+-----------+------------------+
|     168398|     168398|            168398|
|       NULL|       NULL|1963.4289124573927|
|       NULL|       NULL|30.586766032145377|
|USC00300015|       PRCP|              1871|
|USW00094794|   TOBS_s20|              2013|
+-----------+-----------+------------------+



#### groupby and agg
The method `.groupby(col)` groups rows according the value of the column `col`.  
The method `.agg(spec)` computes a summary for each group as specified in `spec`

In [9]:
df.groupby('Measurement').agg({'Year': 'min', 'Station':  'count'}).show()

+-----------+--------------+---------+
|Measurement|count(Station)|min(Year)|
+-----------+--------------+---------+
|   TMIN_s20|         13442|     1873|
|       TMIN|         13442|     1873|
|   SNOW_s20|         15629|     1884|
|       TOBS|         10956|     1876|
|   SNWD_s20|         14617|     1888|
|   PRCP_s20|         16118|     1871|
|   TOBS_s20|         10956|     1876|
|       TMAX|         13437|     1873|
|       SNOW|         15629|     1884|
|   TMAX_s20|         13437|     1873|
|       SNWD|         14617|     1888|
|       PRCP|         16118|     1871|
+-----------+--------------+---------+



In [10]:
# THis command will load the python module that defines the SQL functions
#%load ls ~/spark-latest/python/pyspark/sql/functions.py

### Using SQL queries on DataFrames

There are two main ways to manipulate  DataFrames:

#### Imperative manipulation
Using python methods such as `.select` and `.groupby`.
* Advantage: order of operations is specified.
* Disrdavantage : You need to describe both **what** is the result you want and **how** to get it.

#### Declarative Manipulation (SQL)
* Advantage: You need to describe only **what** is the result you want.
* Disadvantage: SQL does not have primitives for common analysis operations such as **covariance**

### Using sql commands on a dataframe
Spark supports a [subset](https://spark.apache.org/docs/latest/sql-programming-guide.html#supported-hive-features) of the Hive SQL query language.

For example, You can use [Hive `select` syntax](https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Select) to select a subset of the rows in a dataframe.

To use sql on a dataframe you need to first `register` it as a `TempTable`.

for variety, we are using here a small dataframe loaded from a JSON file.

### Counting the number of occurances of each measurement, imparatively

In [11]:
%%time
L=df.groupBy('measurement').count().collect()
#L is a list of Rows (collected DataFrame)

CPU times: user 5.34 ms, sys: 2.28 ms, total: 7.62 ms
Wall time: 1.03 s


In [12]:
D=[(e.measurement,e['count']) for e in L]
print('The most common mesurements')
sorted(D,key=lambda x:x[1], reverse=True)[:6]

The most common mesurements


[('PRCP_s20', 16118),
 ('PRCP', 16118),
 ('SNOW_s20', 15629),
 ('SNOW', 15629),
 ('SNWD_s20', 14617),
 ('SNWD', 14617)]

In [13]:
print('The most rare mesurements')
sorted(D,key=lambda x:x[1], reverse=False)[:6]

The most rare mesurements


[('TOBS', 10956),
 ('TOBS_s20', 10956),
 ('TMAX', 13437),
 ('TMAX_s20', 13437),
 ('TMIN_s20', 13442),
 ('TMIN', 13442)]

### Counting the number of occurances of each measurement, declaratively.

#### Registrering a dataframe as a table in a database

In order to apply SQL commands to a dataframe, it has to first be registered as a table in the database managed by sqlContext.

In [14]:
sqlContext.registerDataFrameAsTable(df,'weather') #using older sqlContext instead of newer (V2.0) sparkSession

In [15]:
%%time
query="""
SELECT measurement,COUNT(measurement) AS count,
                   MIN(year) AS MinYear 
FROM weather  
GROUP BY measurement 
ORDER BY count DESC
"""
print(query)
sqlContext.sql(query).show(5)


SELECT measurement,COUNT(measurement) AS count,
                   MIN(year) AS MinYear 
FROM weather  
GROUP BY measurement 
ORDER BY count DESC

+-----------+-----+-------+
|measurement|count|MinYear|
+-----------+-----+-------+
|   PRCP_s20|16118|   1871|
|       PRCP|16118|   1871|
|   SNOW_s20|15629|   1884|
|       SNOW|15629|   1884|
|   SNWD_s20|14617|   1888|
+-----------+-----+-------+
only showing top 5 rows

CPU times: user 2.53 ms, sys: 3.47 ms, total: 6 ms
Wall time: 1.06 s


#### Performing a map command
* Dataframes do not support `map` and `reduce` operations.
* In order to perform a `map` or `reduce` on a dataframe, you first need to transform it into an RDD.

* This is a quick-and-dirty solution. 
* A better way is to use [built-in sparkSQL functions](https://spark.apache.org/docs/latest/api/sql/index.html)
* Or if you can't find what you need, you can try and create a [User-Defined-function* (UDF)](https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html)

In [16]:
df.rdd.map(lambda row:(row.Station,row.Year)).take(5)

[('USW00094704', 1945),
 ('USW00094704', 1946),
 ('USW00094704', 1947),
 ('USW00094704', 1948),
 ('USW00094704', 1949)]

#### Aggregations 
* **Aggregation** can be used, in combination with built-in sparkSQL functions 
to compute statistics of a dataframe.
* computation will be fast thanks to combined optimzations with database operations.

* A partial list : `count(), approx_count_distinct(), avg(), max(), min()`

* Of these, the interesting one is `approx_count_distinct()` which uses sampling to get an approximate count fast.

In [17]:
import pyspark.sql.functions as F # used here just for show.

In [18]:
df.agg({'station':'approx_count_distinct'}).show()

+------------------------------+
|approx_count_distinct(station)|
+------------------------------+
|                           339|
+------------------------------+



#### Approximate Quantile

* Suppose we want to partition the years into 10 ranges
* such that in each range we have approximately the same number of records.
* The method `.approxQuantile` will use a sample to do this for us.

In [19]:
%%time
print('with accuracy 0.1: ',df.approxQuantile('Year', [0.1*i for i in range(1,10)], 0.1))

with accuracy 0.1:  [1871.0, 1926.0, 1947.0, 1957.0, 1966.0, 1969.0, 1980.0, 1989.0, 2013.0]
CPU times: user 12.2 ms, sys: 3.98 ms, total: 16.2 ms
Wall time: 1.39 s


In [20]:
%%time
print('with accuracy 0.001: ',df.approxQuantile('Year', [0.1*i for i in range(1,10)], 0.00001))

with accuracy 0.001:  [1917.0, 1937.0, 1949.0, 1957.0, 1966.0, 1975.0, 1984.0, 1993.0, 2003.0]
CPU times: user 11.6 ms, sys: 3.24 ms, total: 14.8 ms
Wall time: 1.09 s


### Reading rows selectively from Parquet
Suppose we are only interested in snow measurements. We can apply an SQL query directly to the 
parquet files. As the data is organized in columnar structure, we can do the selection efficiently without loading the whole file to memory.

Here the file is small, but in real applications it can consist of hundreds of millions of records. In such cases loading the data first to memory and then filtering it is very wasteful.

In [21]:
query="""SELECT station,measurement,year 
FROM parquet.`%s` 
WHERE measurement=\"SNOW\" """%weather_parquet
print(query)
df2 = sqlContext.sql(query)
print(df2.count(),df2.columns)
df2.show(5)

SELECT station,measurement,year 
FROM parquet.`NY.parquet` 
WHERE measurement="SNOW" 
15629 ['station', 'measurement', 'year']
+-----------+-----------+----+
|    station|measurement|year|
+-----------+-----------+----+
|USC00308600|       SNOW|1932|
|USC00308600|       SNOW|1956|
|USC00308600|       SNOW|1957|
|USC00308600|       SNOW|1958|
|USC00308600|       SNOW|1959|
+-----------+-----------+----+
only showing top 5 rows



## Summary

* Dataframes can be manipulated decleratively, which allows for more optimization.
* Dataframes can be stored and retrieved from Parquet files.
* It is possible to refer directly to a parquet file in an SQL query.
* See you next time!

## References
* For an introduction to Spark SQL and Dataframes see: [Spark SQL, DataFrames](https://spark.apache.org/docs/latest/sql-programming-guide.html#spark-sql-dataframes-and-datasets-guide)
* Also [spark-dataframe-and-operations](https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/) from [analyticsvidhya.com](https://www.analyticsvidhya.com)

For complete API reference see
* [SQL programming guide](https://spark.apache.org/docs/latest/sql-programming-guide.html) For Java, Scala and Python (Implementation is first in Scala and Python, later pyspark)
* [pyspark API for the DataFrame class](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame) 
* [pyspark API for the pyspark.sql module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module)
